In [31]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.utils import shuffle
from IPython.display import clear_output
from datetime import datetime
import pandas as pd
df=pd.read_csv('sample_images/fer2013.csv', sep=',',header=0)
df_train=df[df['Usage']=='Training']
df_test=df[df['Usage']!='Training']
del df

def str2image(x):
    return np.fromstring(x,sep=' ')

def convertFlat2Image(df):
    mat=df['pixels'].as_matrix()
    r_mat=np.array([str2image(xi).reshape(48,48,1) for xi in mat])
    return r_mat

X_train=convertFlat2Image(df_train)
X_test=convertFlat2Image(df_test)
y_train=df_train['emotion'].as_matrix()
y_test=df_test['emotion'].as_matrix()

Xtrain=X_train[:28000,]/255
Ytrain=y_train[:28000]
Xtest=X_test[:7000,]/255
Ytest=y_test[:7000]


In [63]:
def ImageAug(Xtrain,Ytrain,aug_size=500,critical_val=1.5):
    from keras.preprocessing.image import ImageDataGenerator
    import collections
    import numpy as np
    print("Input count :")
    print(collections.Counter(y_train))
    cat_dict=dict(collections.Counter(y_train))
    max_cat=max(cat_dict.values())
    Xtrain_aug = np.empty_like (Xtrain)
    Xtrain_aug[:] = Xtrain
    Ytrain_aug = np.empty_like (Ytrain)
    Ytrain_aug[:] = Ytrain
    for k in cat_dict:
        total_len=len(Xtrain_aug[Ytrain_aug==k])
        Xtrain_extra=np.empty((0,48,48,1))
        Ytrain_extra=np.empty(0)
        X_k=Xtrain[Ytrain==k]
        Y_k=Ytrain[Ytrain==k]
        datagen = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True)
        datagen.fit(X_k)
        for X_batch, y_batch in datagen.flow(X_k,Y_k, batch_size=aug_size):
            Xtrain_extra=np.append(Xtrain_extra,X_batch,axis=0)
            Ytrain_extra=np.append(Ytrain_extra,y_batch,axis=0)
            total_len=total_len+aug_size
            if max_cat/total_len<critical_val:
                Xtrain_aug=np.append(Xtrain_aug,Xtrain_extra,axis=0)
                Ytrain_aug=np.append(Ytrain_aug,Ytrain_extra,axis=0)
                del Xtrain_extra
                del Ytrain_extra
                break
    print("Output count :")
    print(collections.Counter(Ytrain_aug))
    return Xtrain_aug,Ytrain_aug

    

In [64]:
Xtrain_a,Ytrain_a=ImageAug(Xtrain,Ytrain,aug_size=500,critical_val=1.5)

Input count :
Counter({3: 7215, 6: 4965, 4: 4830, 2: 4097, 0: 3995, 5: 3171, 1: 436})
Output count :
Counter({3.0: 7562, 6.0: 5330, 4.0: 5217, 5.0: 5083, 2.0: 4995, 0.0: 4886, 1.0: 4270})
